# **CASO 3. NIVEL 5 API**

## Básico

In [23]:
#importamos librerías para el análisis exploratorio básico
import numpy as np
import pandas as pd

In [24]:
#agregamos nuestro archivo con la información
df = pd.read_csv("cleaned.csv")

#vemos una muestra de los datos
df.head()

,track_id,genre_top,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,153,Rock,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018
1,154,Rock,0.970135,0.352946,0.023852,0.957113,0.113261,0.032177,53.758,0.035632
2,155,Rock,0.981657,0.142249,0.912122,0.967294,0.363510,0.087527,91.912,0.034325
3,169,Rock,0.989141,0.225978,0.722835,0.263076,0.092371,0.053406,94.322,0.028347
4,170,Rock,0.886660,0.298518,0.744333,0.920950,0.139587,0.088781,97.880,0.073548


In [25]:
#imprimimos el número de filas y columnas de nuestro dataframe
print('El numero de filas es: ', df.shape[0])
print('El numero de columnas es: ', df.shape[1])

El numero de filas es:  4802
El numero de columnas es:  10


In [26]:
#También podemos utilizar la función info() para saber filas y columnas, además de tipos de datos y cantidad de valores nulos por columna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4802 entries, 0 to 4801
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          4802 non-null   int64  
 1   genre_top         4802 non-null   object 
 2   acousticness      4802 non-null   float64
 3   danceability      4802 non-null   float64
 4   energy            4802 non-null   float64
 5   instrumentalness  4802 non-null   float64
 6   liveness          4802 non-null   float64
 7   speechiness       4802 non-null   float64
 8   tempo             4802 non-null   float64
 9   valence           4802 non-null   float64
dtypes: float64(8), int64(1), object(1)
memory usage: 375.3+ KB


Con lo anterior notamos que no hay valores nulos pero eso igual lo podemos confirmar de la siguiente forma:

In [27]:
df.isnull().sum()

track_id            0
genre_top           0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
speechiness         0
tempo               0
valence             0
dtype: int64

Para calcular la media, máximo, mínimo y desviación estándar de las columnas con valores numéricos podemos usar la función describe() que nos dará todos esos datos con una sola instrucción

In [28]:
df.describe()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
count,4802.000000,4.802000e+03,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000
mean,30164.871720,4.870600e-01,0.436556,0.625126,0.604096,0.187997,0.104877,126.687944,0.453413
std,28592.013796,3.681396e-01,0.183502,0.244051,0.376487,0.150562,0.145934,34.002473,0.266632
min,2.000000,9.491000e-07,0.051307,0.000279,0.000000,0.025297,0.023234,29.093000,0.014392
25%,7494.250000,8.351236e-02,0.296047,0.450757,0.164972,0.104052,0.036897,98.000750,0.224617
50%,20723.500000,5.156888e-01,0.419447,0.648374,0.808752,0.123080,0.049594,124.625500,0.446240
75%,44240.750000,8.555765e-01,0.565339,0.837016,0.915472,0.215151,0.088290,151.450000,0.666914
max,124722.000000,9.957965e-01,0.961871,0.999768,0.993134,0.971392,0.966177,250.059000,0.983649


## Intermedio

In [29]:
#importamos las librerías necesarias para el árbol de desiciones
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Definimos la variable que queremos predecir y las variables que nos ayudarán a predecirla, en este caso tomaremos todos los datos de nuestro df excepto las columnas track_id y genre_top para predecir, ya que el id no nos sirve para predecir y el genre_top es el dato a predecir

In [30]:
X = df.drop(['track_id', 'genre_top'], axis=1)
y = df.genre_top

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y)
tree = DecisionTreeClassifier(random_state=42)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Ahora creamos la clase y definimos las variables de acuerdo a las columnas del df

In [31]:
from pydantic import BaseModel

class song(BaseModel):
  acousticness: float 
  danceability: float 
  energy: float 
  instrumentalness: float 
  liveness: float 
  speechiness: float 
  tempo: float 
  valence: float
  class Config:
    schema_extra = { 
      "example": {
        #datos de ejemplo
        "acousticness": 4.87060, 
        "danceability": 0.436556, 
        "energy": 0.669215,
        "instrumentalness": 0.604096,
        "liveness": 0.187997,
        "speechiness": 0.104877,
        "tempo": 126.687944,
        "valence": 0.453413
      }
    }

## Avanzado

In [32]:
#guardamos el modelo como un archivo binario
import pickle
Pkl_Filename = "model_tree.pkl" 
with open(Pkl_Filename, 'wb') as file:
    pickle.dump(tree, file) 

#instalamos fastapi y librería 
!pip install fastapi
from fastapi import FastAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Enseguida hacemos las configuraciones correspondientes para poder utilizar FastAPI y desplegar nuestro modelo

In [41]:
#definimos la variable a utilizar con FastAPI
app = FastAPI()

@app.on_event("startup")

#cargamos nuestro archivo y lo leemos con una instrucción de pickle
def load_model():
    global model
    model = pickle.load(open("model_tree.pkl", "rb"))

#creamos la función que enviará un mensaje al ingresar a la API
@app.get('/')
def index():
    return {'PUNTO DE ACCESO API -- Elizabeth Carrillo'}

#creamos la función que se encargará de tomar la información y realizar la predicción de acuerdo a la clase song definida anteriormente
@app.post('/predict')
def get_music_category(data: song):
    received = data.dict()
    acousticness = received['acousticness']
    danceability = received['danceability'] 
    energy = received['energy'] 
    instrumentalness = received['instrumentalness'] 
    liveness = received['liveness'] 
    speechiness = received['speechiness'] 
    tempo = received['tempo'] 
    valence = received['valence']  
    pred_name = model.predict([[acousticness, danceability, energy, instrumentalness, liveness, speechiness, tempo, valence]]).tolist()[0]
    return {'prediction': pred_name}

In [35]:
#instalamos las librerías necesarias para utilizar la API
!pip install pyngrok
!pip install ngrok
!pip install uvicorn
!pip install nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
#añadimos nuestro token de ngrok
! ngrok authtoken 2DMuKgMtAAWQYkD3rD6YrmD3VIw_6DbfHA9B1jMM4WQ9Hbmsx

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [42]:
from pyngrok import ngrok
import uvicorn
import nest_asyncio

#Creamos el túnel de conexión.
ngrok_tunnel = ngrok.connect(8000)

#Mostrar el URL para acceder.
print('Public URL:', ngrok_tunnel.public_url)

#Conectar la API
nest_asyncio.apply()
uvicorn.run(app, port=8000)
#al ejecutar estos comandos nos dará la URL para acceder a la API

Public URL: http://f6b8-34-122-87-96.ngrok.io


INFO:     Started server process [61]
INFO:uvicorn.error:Started server process [61]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET / HTTP/1.1" 200 OK
INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2806:101e:a:2d76:8121:44aa:3fb0:3d75:0 - "GET /openapi.json HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [61]
INFO:uvicorn.error:Finished server process [61]


In [43]:
ngrok.kill()